<a href="https://colab.research.google.com/github/stevenpl98/bluebikes/blob/master/BlueBikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#For Colab -- DO NOT RUN LOCALLY
import os
#Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Set JAVA_HOME path variable in Linux
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version


#Install Spark
#download Spark file
!wget -q http://apache.osuosl.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz 
#extract the file
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
#Set SPARK-HOME path variable in Linux
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


#install findspark package
!pip install -q findspark

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import findspark
findspark.init()

In [ ]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("bluebikes").setMaster("local[*]")#local[2]
sc=SparkContext(conf = conf)
#spark = SparkSession(sparkContext=sc)

**Add Title Here**

In [ ]:
rdd = sc.textFile('./drive/My Drive/Big Data/data/Bikes Data/Years/2015.csv')
#rdd = sc.textFile('./drive/My Drive/Big Data/data/Bikes Data/Years/hubway_Trips_2013.csv')
rdd.take(3)

In [ ]:
#2019-11-01 00:00:00.2770
test = datetime.strptime('2019-11-01 00:00:00.2770','%Y-%m-%d %H:%M:%S.%f')
print(test)

In [ ]:
mappedrdd = rdd.map(lambda x:x.split(','))

test1 = mappedrdd.map(lambda x: x[1])

temp = test1.first()
test2 = test1.filter(lambda x: x != temp)

test3 = test2.map(lambda x: x[5:7]).distinct()

out = test3.collect()

print(out)

In [ ]:
#12/31/2014 23:58
import time
start = time.time()
from datetime import datetime

def try_parsing_date(text):
    for fmt in ('%Y-%m-%d %H:%M:%S', '%Y-%m-%d %H:%M:%S.%f','%m/%d/%Y %H:%M'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    raise ValueError('String "'+ text +'" could not convert to Datetime')

In [ ]:
#Extracting Years, then going to begin separating by month


rdd = sc.textFile('./drive/My Drive/Big Data/data/Bikes Data/cleaned_data.csv')
mappedrdd = rdd.map(lambda x:x.split(','))

#cleanedrdd = mappedrdd.filter(lambda x: (x[0] != 'Duration' or x[0] != 'tripduration') )
mappedrdd.take(3)
#dates = mappedrdd.map(lambda x: (x[1], x[6]))#.filter(lambda x: (x != 'Start date')).filter(lambda x: (x != 'startdate'))
#header = temp.first()

#dates = temp.filter(lambda x: x[0] != '')#.filter(lambda x: '2014' in x)
#dates = temp.filter(lambda x: (x != 'starttime'))

#dates.take(3)




In [ ]:
years = dates.map(lambda x: try_parsing_date(x).year).distinct()
#years = dates.map(lambda x: x[6:10])
#years.take(3)
out = years.collect()

print(out)

#print(out)
#print(str(time.time() - start) + ' Seconds')

In [ ]:
def RemoveHeader(rdd):
  header = rdd.first()
  data = rdd.filter(lambda x: x != header) 
  return (data)

**Cleaning RDDS**

In [ ]:
# Clean rdds

rdd2011 = rdd = sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/hubway_Trips_2011.csv')
rdd2012 = rdd = sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/hubway_Trips_2012.csv')
rdd2013 = rdd = sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/hubway_Trips_2013.csv')
rdd2014 = rdd = sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2014.csv')
rdd2015 = rdd = sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2015.csv')
rdd2016 = rdd = sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2016.csv')
rdd2017 = rdd = sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2017.csv')
rdd2018 = rdd = sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2018.csv')
rdd2019 = rdd = sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2019.csv')
rdd2020 = rdd = sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2020.csv')

cleaned_rdd = sc.union([RemoveHeader(rdd2011),RemoveHeader(rdd2012),RemoveHeader(rdd2013),RemoveHeader(rdd2014),RemoveHeader(rdd2015),
                       RemoveHeader(rdd2016),RemoveHeader(rdd2017),RemoveHeader(rdd2018),RemoveHeader(rdd2019),RemoveHeader(rdd2020)])
cleaned_rdd.saveAsTextFile('/content/drive/My Drive/Big Data/data/Bikes Data/cleaned_data.csv')


In [ ]:
import pandas as pd
import time
start = time.time()
#out = pd.DataFrame(cleaned_rdd.collect())
#out = cleaned_rdd.collect()
#print(str(time.time() - start))
#out.to_csv('/content/drive/My Drive/Big Data/data/Bikes Data/cleaned_data.csv')

**Month vs Month Difference Throughout Years**

In [ ]:
# clean year rdds
rdd11 =  RemoveHeader(sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/hubway_Trips_2011.csv')).map(lambda x:x.split(','))
rdd12 =  RemoveHeader(sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/hubway_Trips_2012.csv')).map(lambda x:x.split(','))
rdd13 =  RemoveHeader(sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/hubway_Trips_2013.csv')).map(lambda x:x.split(','))
rdd14 =  RemoveHeader(sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2014.csv')).map(lambda x:x.split(','))
rdd15 =  RemoveHeader(sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2015.csv')).map(lambda x:x.split(','))
rdd16 =  RemoveHeader(sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2016.csv')).map(lambda x:x.split(','))
rdd17 =  RemoveHeader(sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2017.csv')).map(lambda x:x.split(','))
rdd18 =  RemoveHeader(sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2018.csv')).map(lambda x:x.split(','))
rdd19 =  RemoveHeader(sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2019.csv')).map(lambda x:x.split(','))
rdd20 =  RemoveHeader(sc.textFile('/content/drive/My Drive/Big Data/data/Bikes Data/Years/2020.csv')).map(lambda x:x.split(','))

In [ ]:
# Month Collection formatting
trips11 = rdd11.map(lambda x: x[1][0:2])
trips12 = rdd12.map(lambda x: x[1][0:2])
trips13 = rdd13.map(lambda x: x[1][0:2])
trips14 = rdd14.map(lambda x: x[1][0:2])
trips15 = rdd15.map(lambda x: x[1][5:7])
trips16 = rdd16.map(lambda x: x[1][5:7])
trips17 = rdd17.map(lambda x: x[1][5:7])
trips18 = rdd18.map(lambda x: x[1][5:7])
trips19 = rdd19.map(lambda x: x[1][5:7])
trips20 = rdd20.map(lambda x: x[1][5:7])

In [ ]:
# Month collections
t11 = trips11.collect()
t12 = trips12.collect()
t13 = trips13.collect()
t14 = trips14.collect()
t15 = trips15.collect()
t16 = trips16.collect()
t17 = trips17.collect()
t18 = trips18.collect()
t19 = trips19.collect()
t20 = trips20.collect()

In [ ]:
# Gets trip numbers per month for all months in a year
def tripsPerMonth(collection):
  trips = pd.DataFrame(collection, columns=["x"]).groupby('x').size().to_dict()
  return trips

In [ ]:
# Format months as example ahead: '1/' or '01' to January
def formatMonth(d):
  changes = {
      '1/':'January',
      '01':'January',
      '2/':'February',
      '02':'February',
      '3/':'March',
      '03':'March',
      '4/':'April',
      '04':'April',
      '5/':'May',
      '05':'May',
      '6/':'June',
      '06':'June',
      '7/':'July',
      '07':'July',
      '8/':'August',
      '08':'August',
      '9/':'September',
      '09':'September',
      '10':'October',
      '11':'November',
      '12':'December'
  }
  d2 = dict((changes[key], value) for (key, value) in d.items())

  return d2

In [ ]:
trips2011 = formatMonth(tripsPerMonth(t11))
trips2012 = formatMonth(tripsPerMonth(t12))
trips2013 = formatMonth(tripsPerMonth(t13))
trips2014 = formatMonth(tripsPerMonth(t14))
trips2015 = formatMonth(tripsPerMonth(t15))
trips2016 = formatMonth(tripsPerMonth(t16))
trips2017 = formatMonth(tripsPerMonth(t17))
trips2018 = formatMonth(tripsPerMonth(t18))
trips2019 = formatMonth(tripsPerMonth(t19))
trips2020 = formatMonth(tripsPerMonth(t20))
print(trips2011)
print(trips2012)
print(trips2013)
print(trips2014)
print(trips2015)
print(trips2016)
print(trips2017)
print(trips2018)
print(trips2019)
print(trips2020)